# Top tokens for different categories

## Read mapping information

In [10]:
import os

import pandas as pd


demoTrainDf = pd.read_csv(os.environ['EHR_DATA_BASE'] + '/blood_pos_cohort_20240614/data/wb_365_wa_1/splits_v1/demographics/mortality_normal_train.csv', sep='\t')
demoTestDf = pd.read_csv(os.environ['EHR_DATA_BASE'] + '/blood_pos_cohort_20240614/data/wb_365_wa_1/splits_v1/demographics/mortality_normal_test.csv', sep='\t')
demoValidateDf = pd.read_csv(os.environ['EHR_DATA_BASE'] + '/blood_pos_cohort_20240614/data/wb_365_wa_1/splits_v1/demographics/mortality_normal_validate.csv', sep='\t')
demoDf = pd.concat([demoTrainDf, demoTestDf, demoValidateDf], ignore_index=True)
demoDf

,person_id,visit_occurrence_id,TYPE,COUNTRY_OF_BIRTH,SEX,Age_Bin,JOURNEY_ID
0,678487,3312694,Emergency,Australia,Male,"(60, 70]",3312694
1,2094045,9199249,Emergency,Australia,Male,"(20, 30]",9199249
2,2107813,9410987,Emergency,Australia,Male,"(50, 60]",9410987
3,2083212,602890,Inpatient,Australia,Female,"(40, 50]",9039437
4,2083212,9039437,Emergency,Australia,Female,"(40, 50]",9039437
...,...,...,...,...,...,...,...
7194,2603535,16588204,Inpatient,Australia,Male,"(60, 70]",16588204
7195,2603535,16709341,Inpatient,Australia,Male,"(60, 70]",16709341
7196,2603535,16651755,Inpatient,Australia,Male,"(60, 70]",16651755
7197,2098616,16872774,Emergency,Australia,Female,"(80, 90]",16872774


In [11]:
import os

mappingDf = pd.read_csv(os.environ['GENOMICS_DATA_BASE'] + '/patient_tube_id_mapping_full.tsv', sep='\t')
mappingDf

,tube_code,PATIENT_ID,db_ID,pt_age,pt_gender,EPISODE_ID,hospital_admission,hospital_discharge,hospital_in_last_year,genome_species,species_reported,contig_number,length,tube_code_duplicate_or_old,date_of_collection,ID_number,location_additional_02,collected_from_original
0,AH19J072,213972.0,3JNQS,64,Male,12931474.0,2019-10-17,2019-11-29,yes,Escherichia coli,Escherichia coli,92,5131021,NaN,2019-10-30,19-303-0997,A-7EA;HAEM,Blood
1,AH19J074,213972.0,3JNQS,64,Male,12931474.0,2019-10-17,2019-11-29,yes,Streptococcus sp. D19,Streptococcus mitis group,84,2022991,NaN,2019-10-30,19-303-1174,A-7EA;HAEM,Blood Peripheral
2,AH19B003,526238.0,567AE,85,Male,640261.0,2019-02-04,2019-02-07,no,Escherichia coli,Escherichia coli,93,4942915,NaN,2019-02-03,19-034-1596,A-4WB;GMC,Blood
3,AH20A024,788941.0,3YBDS,58,Female,13194824.0,2020-01-16,2020-01-19,no,Escherichia coli,Escherichia coli,110,5149157,NaN,2020-01-16,20-016-2112,S-ED;Emergency,Blood Venous
4,AH20L041,1023735.0,2VJ78,42,Male,14210147.0,2020-12-18,2021-01-05,yes,Staphylococcus aureus,Staphylococcus aureus,46,2735216,NaN,2020-12-16,20-351-2856,Emergency Dept;nan,Blood
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3112,ALF22L076,639020.0,3S6HL,84,Male,17219899.0,2022-12-15,2022-12-20,yes,Streptococcus vestibularis,Streptococcus salivarius group,107,1938627,NaN,2022-12-15,22-349-2150,Ward 4WB,Blood Peripheral
3113,ALF22L089,655404.0,6EWBM,67,Female,NaN,NaN,NaN,NaN,Proteus columbae,Proteus vulgaris,152,3972219,NaN,2022-12-17,22-351-1465,Comm/Amb Clinic,Blood
3114,ALF22L138,2572048.0,YM5PJ,61,Male,NaN,NaN,NaN,NaN,Enterobacter roggenkampii,Enterobacter cloacae complex,66,4876179,NaN,2022-12-27,22-361-0202,Ward 7 East,Hick White Lum
3115,ALF23A102,2054964.0,5DYAH,35,Female,17294287.0,2023-01-12,2023-02-01,yes,Streptococcus constellatus,Streptococcus anginosus group,54,1869828,NaN,2023-01-11,23-011-3305,ICU,Blood Venous


In [12]:
mappedJourneyTubeIdsDf = mappingDf[['tube_code', 'PATIENT_ID', 'EPISODE_ID']].drop_duplicates().merge(
    demoDf[['person_id', 'visit_occurrence_id', 'JOURNEY_ID']],
    how='inner',
    left_on=['PATIENT_ID', 'EPISODE_ID'],
    right_on=['person_id', 'visit_occurrence_id']
).drop(
    columns=['PATIENT_ID', 'EPISODE_ID', 'visit_occurrence_id']
)[['person_id', 'JOURNEY_ID', 'tube_code']].drop_duplicates()
mappedJourneyTubeIdsDf

,person_id,JOURNEY_ID,tube_code
0,50056,13224842,AH20B011
1,62567,12645629,AH19G065
2,138108,14737781,AH21E052
3,211314,13955099,AH20J009
4,211314,13955099,KPN2214
...,...,...,...
512,2084373,16158768,ALF22C004
513,1058066,16759482,ALF22H089
514,2552781,16966609,ALF22J054
515,2638484,16969999,ALF22K143


## Read bed files containing overlapping tokens and attribution scores

In [13]:
import os
import pandas as pd
from pathlib import Path


overlappingFilesDir = Path(os.environ['GENOMICS_DATA_BASE'], 'genome_nlp_tokens', 'overlapping_with_annotations')

overlappingDfList = []
for overlappingFile in os.listdir(overlappingFilesDir):
    df = pd.read_csv(Path(overlappingFilesDir, overlappingFile), sep='\t', names=['contig_id', 'start_position', 'end_position', 'tokens', 'score', 'id', 'name', 'gene', 'atributes'])
    df['tube_code'] = [overlappingFile.split('_')[0]]*df.shape[0]
    overlappingDfList.append(df)
overlappingDf = pd.concat(overlappingDfList, ignore_index=True)
overlappingDf = overlappingDf.merge(
    mappedJourneyTubeIdsDf,
    how='inner',
    on=['tube_code']
)
overlappingDf = overlappingDf[['tube_code', 'contig_id', 'start_position', 'end_position', 'tokens', 'score']].drop_duplicates()
overlappingDf

,tube_code,contig_id,start_position,end_position,tokens,score
0,AH19E065,1,191,192,GT,-0.001426
1,AH19E065,1,193,197,ATAAC,-0.005218
2,AH19E065,1,198,199,GC,-0.000682
3,AH19E065,1,200,203,GCCA,-0.010548
4,AH19E065,1,204,210,ATTGATT,-0.004120
...,...,...,...,...,...,...
121252,AH21K020,31,99,103,GGTGG,-0.252315
121253,AH21K020,31,104,106,TTC,-0.134230
121254,AH21K020,31,107,110,GAGT,-0.080241
121255,AH21K020,31,111,116,CCACTT,0.215479


## Top 20 tokens (Overall)

In [14]:
overlappingDf.tokens.value_counts().reset_index().sort_values(by=['count'], ascending=False)[:20]

,tokens,count
0,CG,1134
1,CGT,526
2,TTC,490
3,CGG,486
4,GC,432
5,ACG,395
6,ACGAT,312
7,ATGAT,307
8,GCAAT,306
9,CCG,301


## Top 20 tokens with high attribution scores (score > 0.02)

In [15]:
overlappingDf[overlappingDf.score > 0.02].tokens.value_counts().reset_index().sort_values(by=['count'], ascending=False)[:20]

,tokens,count
0,CGT,35
1,GCTC,34
2,AGAGC,31
3,CCGG,29
4,TAATTT,29
5,CAC,27
6,ATCC,26
8,GTGC,26
7,CTCAGC,26
9,CG,25


## Top 20 tokens without considering the high attribution scores (score < 0.02)

In [16]:
overlappingDf[overlappingDf.score < 0.02].tokens.value_counts().reset_index().sort_values(by=['count'], ascending=False)[:20]

,tokens,count
0,CG,1109
1,CGT,491
2,TTC,487
3,CGG,483
4,GC,432
5,ACG,394
6,ACGAT,312
7,ATGAT,307
8,GCAAT,306
9,CCG,299


## Top 20 tokens with low attribution scores (score < -0.02)

In [17]:
overlappingDf[overlappingDf.score < -0.02].tokens.value_counts().reset_index().sort_values(by=['count'], ascending=False)[:20]

,tokens,count
0,CG,89
1,CGT,88
2,GC,80
3,CGAA,70
4,TTC,67
5,CCC,66
6,CCTGC,63
7,GAACCC,58
8,GACCCC,58
9,GACC,57


## Top 20 tokens without considering the low attribution scores (score > -0.02)

In [18]:
overlappingDf[overlappingDf.score > -0.02].tokens.value_counts().reset_index().sort_values(by=['count'], ascending=False)[:20]

,tokens,count
0,CG,1045
1,CGG,465
2,CGT,438
3,TTC,423
4,ACG,391
5,GC,352
6,GCAAT,299
7,ACGAT,297
8,CCG,293
9,ATGAT,286
